In [16]:
import torch
import torch.nn as nn
import numpy as np
from torch import optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from sklearn.model_selection import train_test_split
import os
from skimage import io
import time
import ast 

In [17]:
if torch.cuda.is_available:
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [18]:
#train on a subset 
df = pd.read_csv('large_nodules_png_bounding_box.csv')

In [35]:
_ , X_random, = train_test_split(df, test_size=0.01, random_state=0)
X_random.to_csv('large_nodules_png_bounding_box.csv',index=False)

In [37]:
df_subset = pd.read_csv('large_nodules_png_bounding_box.csv')
# torch.FloatTensor(df_subset.iloc[1,3])
df_subset.head()

,path,imageName,SOPInstanceUID,boxes,area
0,/scratch/ebc308/tcia/data_png/LIDC-IDRI/LIDC-I...,000280.png,1.3.6.1.4.1.14519.5.2.1.6279.6001.147041882929...,"[[98.0, 276.0, 104.0, 280.0]]",[24.0]
1,/scratch/ebc308/tcia/data_png/LIDC-IDRI/LIDC-I...,000052.png,1.3.6.1.4.1.14519.5.2.1.6279.6001.131504933114...,"[[388.0, 252.0, 410.0, 269.0]]",[374.0]
2,/scratch/ebc308/tcia/data_png/LIDC-IDRI/LIDC-I...,000080.png,1.3.6.1.4.1.14519.5.2.1.6279.6001.116343254440...,"[[135.0, 324.0, 138.0, 328.0]]",[12.0]
3,/scratch/ebc308/tcia/data_png/LIDC-IDRI/LIDC-I...,000243.png,1.3.6.1.4.1.14519.5.2.1.6279.6001.335140364211...,"[[371.0, 327.0, 388.0, 347.0]]",[340.0]
4,/scratch/ebc308/tcia/data_png/LIDC-IDRI/LIDC-I...,000209.png,1.3.6.1.4.1.14519.5.2.1.6279.6001.108290509405...,"[[54.0, 297.0, 60.0, 311.0]]",[84.0]


In [53]:
temp = df_subset.iloc[9,3]
res = ast.literal_eval(temp) 
res

[[148.0, 369.0, 160.0, 378.0]]

In [25]:
img_name = df_subset.iloc[9,0]
image = io.imread(img_name)
image.shape

(512, 512)

In [149]:
from torchvision import transforms
# torchvision models are trained on input images normalized to [0 1] range .ToPILImage() function achives this
# additional normalization is required see: http://pytorch.org/docs/master/torchvision/models.html


# image needs to be pil image h x w
# mask rcnn needs : https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html
train_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.RandomResizedCrop(896),
#         transforms.RandomHorizontalFlip(),
        transforms.ToTensor()])
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

validation_transform = transforms.Compose([
        transforms.ToPILImage(),
#         transforms.CenterCrop(896),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

class ResNet_Dataloader(Dataset):
    """Chest X-ray dataset from https://nihcc.app.box.com/v/ChestXray-NIHCC."""

    def __init__(self, csv_file, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file filename information.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.data_frame = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        # get image path
        img_name = (self.data_frame.iloc[idx, 0])
        
        # open image 
#         print("image name:" , img_name)
        image = io.imread(img_name)
        
        # unclear what this is doing.. will leave it in for now
        if len(image.shape) > 2 and image.shape[2] == 4:
            image = image[:,:,0]
            
        image = np.repeat(image[None,...],3,axis=0)
        
#         image_class = self.data_frame.iloc[idx, -1]
        
        # transform image? 
        if self.transform:
            image = self.transform(np.uint8(image))
        
        imageID = (self.data_frame.iloc[idx, 2])
        
        #boxes are strings
        boxes = ast.literal_eval(self.data_frame.iloc[idx, 3]) 
        boxes = torch.FloatTensor(boxes) 
#         print(boxes.shape)
        
        areas = ast.literal_eval(self.data_frame.iloc[idx, 4]) 
        areas = torch.FloatTensor(areas) 
            
        target = {'boxes': boxes, 'labels': 1, 'image_id':imageID, 'area': areas, 'iscrowd':False }
        sample = {'x': image, 'y': target}
        print(image.shape)
        return image, target

In [150]:
train_df_path = 'large_nodules_png_bounding_box.csv'

transformed_dataset_Resnet = {'train': ResNet_Dataloader(train_df_path, transform=train_transform)}
# bs = 16
bs = 16 #if bigger, running out of memory in p100
dataloader_Resnet = {x : DataLoader(transformed_dataset_Resnet[x], batch_size=bs,
                        shuffle=True, num_workers=0) for x in ['train']}
data_sizes ={x: len(transformed_dataset_Resnet[x]) for x in ['train']}

In [151]:
sample = next(iter(dataloader_Resnet['train']))
print(sample['x'].shape)
# sample.shape

torch.Size([3, 896, 896])
torch.Size([3, 896, 896])
torch.Size([3, 896, 896])
torch.Size([3, 896, 896])
torch.Size([3, 896, 896])
torch.Size([3, 896, 896])
torch.Size([3, 896, 896])
torch.Size([3, 896, 896])
torch.Size([3, 896, 896])
torch.Size([3, 896, 896])
torch.Size([3, 896, 896])
torch.Size([3, 896, 896])
torch.Size([3, 896, 896])
torch.Size([3, 896, 896])
torch.Size([3, 896, 896])
torch.Size([3, 896, 896])


RuntimeError: invalid argument 0: Sizes of tensors must match except in dimension 0. Got 1 and 2 in dimension 1 at /opt/conda/conda-bld/pytorch_1579022060824/work/aten/src/TH/generic/THTensor.cpp:612

In [144]:
# print(sample['y'])

In [133]:
def train_model(model, dataloader, optimizer, scheduler, loss_fn, num_epochs = 10, verbose = False):
#     acc_dict = {'train':[],'validate':[]}
#     loss_dict = {'train':[],'validate':[]}
    acc_dict = {'train':[]}
    loss_dict = {'train':[]}
    best_acc = 0
#     phases = ['train','validate']
    phases = ['train']
    since = time.time()
    epochLossTrain = list()
    epochAccTrain = list()
#     epochLossValidate = list()
#     epochAccValidate = list()
    for i in range(num_epochs):
        print('Epoch: {}/{}'.format(i, num_epochs-1))
        print('-'*10)
        losslist = []
        for p in phases:
            running_correct = 0
            running_loss = 0
            running_total = 0
            if p == 'train':
                model.train()
#                 model.zero_grad()
            else:
                model.eval()
                
            for data in dataloader[p]:
                optimizer.zero_grad()
                #cpu
                print(data['x'])
                image = data['x']
                target = data['y']
#                 print(label)
                #gpu
#                 image = data['x'].to(device)
#                 label = data['y'].to(device)
                output = model(image, target)
                output = output.squeeze(1)
                output = output.float()
                label = label.float()
                
#                 print(output.shape)
#                 print(label.shape)
                
                loss = loss_fn(output, label)
#                 loss = loss.float()
#                 _, preds = torch.max(output, dim = 1)
                num_imgs = image.size()[0]
#                 print(label)
#                 print(output)
                running_correct += torch.sum(output ==label).item()
                running_loss += loss.item()*num_imgs
                running_total += num_imgs
                if p == 'train':
#                     a = list(model.parameters())[0].clone()
                    loss.backward()
                    optimizer.step()
#                     b = list(model.parameters())[0].clone()
#                     print(torch.equal(a, b))
#                     print(list(model.parameters())[0].grad)
            epoch_acc = float(running_correct/running_total)
            epoch_loss = float(running_loss/running_total)
            #plot epoch loss
            if p == 'train':
                epochLossTrain.append(epoch_loss)
                epochAccTrain.append(epoch_acc)
#             if p == 'validate':
#                 epochLossValidate.append(epoch_loss)
#                 epochAccValidate.append(epoch_acc)
            if verbose or (i%10 == 0):
                print('Phase:{}, epoch loss: {:.4f} Acc: {:.4f}'.format(p, epoch_loss, epoch_acc))
            acc_dict[p].append(epoch_acc)
            loss_dict[p].append(epoch_loss)      
#             if p == 'validate':
#                 if epoch_acc > best_acc:
#                     best_acc = epoch_acc
#                     best_model_wts = model.state_dict()
#             else:
            if scheduler:
                scheduler.step()
        
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val acc: {:4f}'.format(best_acc))
    
#     model.load_state_dict(best_model_wts)
#     return model, acc_dict, loss_dict, epochLossTrain, epochLossValidate, epochAccTrain, epochAccValidate
    return model, acc_dict, loss_dict, epochLossTrain, epochAccTrain


In [134]:
import torchvision.models as models

maskrcnn = models.detection.maskrcnn_resnet50_fpn(pretrained=True)

# # resnet18 = models.resnet18().to(device)
# resnet18 = models.resnet18()
# new_model = nn.Sequential(*list(resnet18.children())[:-1])
# resnet18
# # output = resnet18(sample['x'].to(device))
# output = resnet18(sample['x'])
# output.shape

In [135]:
# maskrcnn

In [136]:
# resnet18.fc = nn.Linear(512, 4) # assuming that the fc7 layer has 512 neurons, otherwise change it 
# # resnet18.
# # resnet18
# resnet18 = resnet18

In [137]:
# output = resnet18(sample['x'].to(device))
# output = resnet18(sample['x'])
# output.shape
images,targets = next(iter(dataloader_Resnet['train']))
images = list(image for image in images)
targets = [{k: v for k, v in t.items()} for t in targets]
output = model(images,targets)  

# output = maskrcnn(sample['x'])
# output.shape

RuntimeError: invalid argument 0: Sizes of tensors must match except in dimension 0. Got 1 and 2 in dimension 1 at /opt/conda/conda-bld/pytorch_1579022060824/work/aten/src/TH/generic/THTensor.cpp:612

In [126]:
# optimizer = optim.Adam(resnet18.parameters(), lr = 0.001)
optimizer = optim.Adam(maskrcnn.parameters(), lr = 0.001)

cel = nn.MSELoss() 
# cel = nn.NLLLoss() 
lambda_func = lambda epoch: 0.5 ** epoch
scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda_func)

In [127]:
maskrcnn, acc_dict, loss_dict, epochLossTrain, epochAccTrain = train_model(maskrcnn, dataloader_Resnet, optimizer, 
                                                                           scheduler, cel, num_epochs=10, verbose = True)


Epoch: 0/9
----------
tensor([[[[-1.5870, -1.5870, -1.5870,  ..., -1.6727, -1.6727, -1.6727],
          [-1.5870, -1.5870, -1.5870,  ..., -1.6727, -1.6727, -1.6727],
          [-1.5870, -1.5870, -1.5870,  ..., -1.6727, -1.6727, -1.6727],
          ...,
          [-1.8268, -1.8268, -1.8268,  ..., -1.4500, -1.4500, -1.4500],
          [-1.8268, -1.8268, -1.8268,  ..., -1.4500, -1.4500, -1.4500],
          [-1.8268, -1.8268, -1.8268,  ..., -1.4500, -1.4500, -1.4500]],

         [[-1.7381, -1.7381, -1.7381,  ..., -1.7031, -1.7031, -1.7031],
          [-1.7381, -1.7381, -1.7381,  ..., -1.7031, -1.7031, -1.7031],
          [-1.7381, -1.7381, -1.7381,  ..., -1.7031, -1.7031, -1.7031],
          ...,
          [-1.8081, -1.8081, -1.8081,  ..., -1.6681, -1.6681, -1.6681],
          [-1.8081, -1.8081, -1.8081,  ..., -1.6681, -1.6681, -1.6681],
          [-1.8081, -1.8081, -1.8081,  ..., -1.6681, -1.6681, -1.6681]],

         [[-1.2816, -1.2816, -1.2816,  ..., -1.1596, -1.1596, -1.1596],
        

KeyError: 0